<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####

import pandas as pd
import numpy as np

df = pd.read_csv('TelcoCustomer.csv',na_values=['', ' '])
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [3]:
df.nunique()

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6530
Churn                  2
dtype: int64

In [4]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,6235-VDHOM,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [5]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [6]:
df["TotalCharges"] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [7]:
df['TotalCharges'].value_counts()

20.20      11
19.75       9
19.65       8
20.05       8
19.90       8
45.30       7
19.55       7
19.45       6
20.15       6
20.25       6
20.30       5
20.45       5
69.95       4
69.90       4
19.85       4
69.60       4
44.40       4
69.65       4
19.50       4
19.20       4
19.95       4
49.90       4
20.50       4
20.35       4
70.60       4
44.00       4
50.15       4
19.40       4
75.30       4
74.70       4
           ..
1971.15     1
1793.25     1
1212.85     1
1504.05     1
1319.95     1
572.20      1
5958.85     1
1673.80     1
958.15      1
7610.10     1
92.05       1
2666.75     1
1001.50     1
7262.00     1
4740.00     1
96.45       1
4483.95     1
5574.75     1
44.65       1
2688.45     1
1374.35     1
3107.30     1
248.95      1
607.30      1
1534.05     1
1066.15     1
249.95      1
8333.95     1
7171.70     1
1024.00     1
Name: TotalCharges, Length: 6530, dtype: int64

In [8]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,2
top,6235-VDHOM,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,5174


In [9]:
df = df.drop(['customerID'], axis=1)

In [10]:
!pip install category_encoders

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=9809)

In [12]:
X_train = train.drop(columns=['Churn'], axis=1)
y_train = train['Churn']
X_test = test.drop(columns=['Churn'], axis=1)
y_test = test['Churn']

In [13]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)

In [14]:
X_train = encoder.fit_transform(X_train)
X_test = encoder.fit_transform(X_test)

In [17]:
y_train.head()

2406    Yes
6403     No
1119     No
358     Yes
4920     No
Name: Churn, dtype: object

In [18]:
y_train = y_train.map({'Yes': 1, 'No': 0})
y_test = y_test.map({'Yes': 1, 'No': 0})

In [20]:
y_train.head()

2406    1
6403    0
1119    0
358     1
4920    0
Name: Churn, dtype: int64

In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train[:10])

[[ 1.01757201 -1.01757201 -0.44100129  0.97199368 -0.97199368  0.65477171
  -0.65477171 -1.11432518  0.32762406 -0.32762406  1.04413867 -0.85911101
  -0.32762406  1.12948803 -0.72323444 -0.52667651  1.0079834  -0.63417411
  -0.52667651  1.38151739 -0.88467662 -0.52667651  1.12905409 -0.72293084
  -0.52667651  1.00874708 -0.63476109 -0.52667651  1.26265694 -0.81205959
  -0.52667651  1.26013772 -0.81045552 -0.52667651  0.9035312  -0.56583563
  -0.51080472  0.83113239 -0.83113239  1.40801128 -0.53458754 -0.51992525
  -0.54599235  1.04732796 -0.78003932]
 [-0.98273143  0.98273143 -0.44100129 -1.02881327  1.02881327 -1.52724985
   1.52724985  1.60713367  0.32762406 -0.32762406 -0.95772719  1.16399392
  -0.32762406  1.12948803 -0.72323444 -0.52667651 -0.99207983  1.57685403
  -0.52667651  1.38151739 -0.88467662 -0.52667651 -0.88569716  1.38325818
  -0.52667651 -0.99132877  1.57539586 -0.52667651  1.26265694 -0.81205959
  -0.52667651  1.26013772 -0.81045552 -0.52667651 -1.10676864  1.76729769

In [23]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [26]:
inputs = X_train.shape[1]
epochs = 75
batch_size = 20

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )

Train on 5282 samples, validate on 1761 samples
Epoch 1/75
5282/5282 [==============================] - 1s 272us/sample - loss: nan - acc: 0.7378 - val_loss: nan - val_acc: 0.7212
Epoch 2/75
5282/5282 [==============================] - 1s 125us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 3/75
5282/5282 [==============================] - 1s 125us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 4/75
5282/5282 [==============================] - 1s 126us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 5/75
5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 6/75
5282/5282 [==============================] - 1s 126us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 7/75
5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 8/75
5282/5282 [======

5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 63/75
5282/5282 [==============================] - 1s 130us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 64/75
5282/5282 [==============================] - 1s 128us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 65/75
5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 66/75
5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 67/75
5282/5282 [==============================] - 1s 126us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 68/75
5282/5282 [==============================] - 1s 127us/sample - loss: nan - acc: 0.7391 - val_loss: nan - val_acc: 0.7212
Epoch 69/75
5282/5282 [==============================] - 1s 126us/sample - loss: 

In [30]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [20]}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.7391139870676656 using {'batch_size': 10, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 10, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 20, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 40, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 60, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 80, 'epochs': 20}
Means: 0.7391139870676656, Stdev: 0.007478484762892845 with: {'batch_size': 100, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?